In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
import sys,os
from tqdm import tqdm_notebook as tqdm

c = 'Kaggle-IEEE-CIS-Fraud-Detection-2019'
root = '/home/daishu/competition/%s/'%c
id_name = 'TransactionID'
label_name = 'isFraud'

In [15]:
train_df1 = pd.read_csv('%s/data/train_identity.csv'%root)
train_df2 = pd.read_csv('%s/data/train_transaction.csv'%root)

test_df1 = pd.read_csv('%s/data/test_identity.csv'%root)
test_df2 = pd.read_csv('%s/data/test_transaction.csv'%root)

In [16]:
train_df1.shape,train_df2.shape

((144233, 41), (590540, 394))

In [17]:
train_df = train_df2.merge(train_df1,how='left',on=id_name)
test_df = test_df2.merge(test_df1,how='left',on=id_name)

In [18]:
del train_df1,train_df2,test_df1,test_df2
train_df.shape

(590540, 434)

In [3]:
train_df.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
test_df.head()

,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,0.0,0.0,310.0,90.0,0.0,310.0,90.0,0.0,0.0,0.0
2,3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
def Information_entropy(x):
    x = pd.Series(x)
    pct = x.groupby(by=x).count() / len(x)
    return np.sum(-pct*np.log2(pct))

def Mutual_information(x,y):
    df = pd.DataFrame({'x':x,'y':y})
    entropy = Information_entropy(y)
    conditiona_entropy = df.groupby(['x']).agg({'y':Information_entropy})
    pct = df.groupby(['x']).count() / len(x)
    return entropy - np.sum(pct.values*conditiona_entropy.values)

def Data_review(trainDf,testDf,idName,labelName):
    info = []
    info.append([idName,'object',trainDf.shape[0],testDf.shape[0],np.nan,np.nan,np.nan,np.nan,np.nan])
    info.append([labelName,trainDf[labelName].dtype,trainDf.shape[0],testDf.shape[0],trainDf[labelName].mean(),np.nan,np.nan,np.nan,np.nan])
    for col in tqdm(trainDf.columns):
        if col in [idName,labelName]:
            continue
        data_type = trainDf[col].dtype
        train_nunique = trainDf[col].nunique()
        test_nunique = testDf[col].nunique()
        if data_type == 'object':
            train_mode_or_mean = trainDf[col].mode().values[0]
            test_mode_or_mean = testDf[col].mode().values[0]
            mi_or_corr = Mutual_information(trainDf[col],trainDf[labelName])
        else:
            train_mode_or_mean = trainDf[col].mean()
            test_mode_or_mean = testDf[col].mean()
            mi_or_corr = trainDf[col].corr(trainDf[labelName])
        train_nan_num = trainDf[col].isnull().sum()
        test_nan_num = testDf[col].isnull().sum()
        info.append([col,data_type,train_nunique,test_nunique,train_mode_or_mean,test_mode_or_mean,train_nan_num,test_nan_num,mi_or_corr])
    info_df = pd.DataFrame(info,columns=['column','dataType','trainNunique','testNunique','trainModeOrMean','testModeOrMean','trainNanNum','testNanNum','miOrCorr'])
    return info_df

In [22]:
info_df = Data_review(train_df,test_df,id_name,label_name)

In [23]:
info_df

,column,dataType,trainNunique,testNunique,trainModeOrMean,testModeOrMean,trainNanNum,testNanNum,miOrCorr
0,TransactionID,object,590540,506691,NaN,NaN,NaN,NaN,NaN
1,isFraud,int64,590540,506691,0.03499,NaN,NaN,NaN,NaN
2,TransactionDT,int64,573349,494686,7.37231e+06,2.69299e+07,0.0,0.0,0.013103
3,TransactionAmt,float64,20902,14119,135.027,134.726,0.0,0.0,0.011320
4,ProductCD,object,5,5,W,W,0.0,0.0,0.014839
5,card1,int64,13553,13244,9898.73,9957.22,0.0,0.0,-0.013640
6,card2,float64,500,497,362.555,363.735,8933.0,8654.0,0.003388
7,card3,float64,114,115,153.195,153.543,1565.0,3002.0,0.154151
8,card4,object,4,4,visa,visa,1577.0,3086.0,0.000809
9,card5,float64,119,102,199.279,200.163,4259.0,4547.0,-0.033580


In [71]:
info_df.to_csv('%s/data/info.csv'%root,index=False)

In [11]:
train_df['id_03'].nunique()

24

In [53]:
card_info = ['card%s'%(i+1) for i in range(6)]
card_info.extend(['addr1','addr2'])
tmp = train_df[card_info]

In [54]:
for col in tmp.columns:
    tmp[col] = tmp[col].astype(str).apply(lambda x:x.replace('.0',''))


In [55]:
tmp['card'] = tmp[card_info].apply(lambda x:'_'.join(x),axis=1)
tmp

,card1,card2,card3,card4,card5,card6,addr1,addr2,card
0,13926,nan,150,discover,142,credit,315,87,13926_nan_150_discover_142_credit_315_87
1,2755,404,150,mastercard,102,credit,325,87,2755_404_150_mastercard_102_credit_325_87
2,4663,490,150,visa,166,debit,330,87,4663_490_150_visa_166_debit_330_87
3,18132,567,150,mastercard,117,debit,476,87,18132_567_150_mastercard_117_debit_476_87
4,4497,514,150,mastercard,102,credit,420,87,4497_514_150_mastercard_102_credit_420_87
5,5937,555,150,visa,226,debit,272,87,5937_555_150_visa_226_debit_272_87
6,12308,360,150,visa,166,debit,126,87,12308_360_150_visa_166_debit_126_87
7,12695,490,150,visa,226,debit,325,87,12695_490_150_visa_226_debit_325_87
8,2803,100,150,visa,226,debit,337,87,2803_100_150_visa_226_debit_337_87
9,17399,111,150,mastercard,224,debit,204,87,17399_111_150_mastercard_224_debit_204_87


In [56]:
tmp['card'].nunique()

43071

In [60]:
tmp[label_name] = train_df[label_name]

In [63]:
tmp.groupby(['card']).agg({label_name:['count','mean']}).sort_values(by=[(label_name,'count')],ascending=False)

isFraud          
                                                   count      mean
card                                                              
15885_545_185_visa_138_debit_nan_nan                9900  0.043737
17188_321_150_visa_226_debit_299_87                 5862  0.023883
12695_490_150_visa_226_debit_325_87                 5766  0.031217
9500_321_150_visa_226_debit_204_87                  4647  0.026899
3154_408_185_mastercard_224_debit_nan_nan           4398  0.062301
12839_321_150_visa_226_debit_264_87                 3538  0.010458
16132_111_150_visa_226_debit_299_87                 3523  0.014476
15497_490_150_visa_226_debit_299_87                 3419  0.010822
9500_321_150_visa_226_debit_272_87                  2715  0.023204
5812_408_185_mastercard_224_debit_nan_nan           2639  0.112543
4461_375_185_mastercard_224_debit_nan_nan           2567  0.075185
7664_490_150_visa_226_debit_264_87                  2548  0.035322
7207_111_150_visa_226_debit_204_87                  2320  0.010776
9112_250_150_visa_226_debit_441_87                  2287  0.014429
9633_130_185_visa_138_debit_nan_nan                 2139  0.215521
12544_321_150_visa_226_debit_441_87                 2104  0.012357
10023_111_150_visa_226_debit_325_87                 2082  0.014890
16136_204_185_visa_138_debit_nan_nan                1974  0.035968
13832_375_185_mastercard_224_debit_nan_nan          1949  0.097999
9026_545_185_visa_137_credit_nan_nan                1942  0.199279
9500_321_150_visa_226_debit_330_87                  1919  0.068786
4806_490_150_visa_226_debit_315_87                  1853  0.012412
9633_296_185_visa_138_debit_nan_nan                 1814  0.141676
17399_111_150_mastercard_224_debit_204_87           1802  0.012209
9500_321_150_visa_226_debit_126_87                  1708  0.008197
12501_490_150_visa_226_debit_204_87                 1600  0.028125
17131_111_150_mastercard_224_debit_264_87           1515  0.009901
17188_321_150_visa_226_debit_310_87                 1475  0.004746
12544_321_150_visa_226_debit_337_87                 1473  0.044807
15775_481_150_mastercard_102_credit_330_87          1453  0.000000
...                                                  ...       ...
1756_383_150_visa_226_debit_143_87                     1  0.000000
17567_555_150_mastercard_137_credit_330_87             1  0.000000
17562_555_185_mastercard_224_credit_216_60             1  1.000000
17560_168_144_visa_137_credit_426_96                   1  0.000000
17560_168_144_visa_137_credit_236_96                   1  0.000000
17583_528_150_visa_226_credit_231_87                   1  0.000000
17583_528_150_visa_226_credit_299_87                   1  0.000000
17607_182_150_visa_226_debit_126_87                    1  0.000000
17598_268_150_visa_166_debit_324_87                    1  0.000000
17604_555_195_mastercard_224_debit_454_69              1  0.000000
17602_555_150_visa_226_debit_476_87                    1  0.000000
17602_555_150_visa_226_debit_204_87                    1  0.000000
17599_216_150_mastercard_102_credit_191_87             1  0.000000
17598_nan_nan_nan_nan_nan_337_87                       1  0.000000
17598_nan_nan_nan_nan_nan_264_87                       1  0.000000
17598_268_150_visa_166_debit_nan_nan                   1  0.000000
17598_268_150_visa_166_debit_420_87                    1  0.000000
17598_268_150_visa_166_debit_387_87                    1  0.000000
17598_268_150_visa_166_debit_253_87                    1  0.000000
17583_528_150_visa_226_credit_330_87                   1  0.000000
17598_268_150_visa_166_debit_220_87                    1  0.000000
17598_268_150_visa_166_debit_177_87                    1  1.000000
17598_268_150_visa_166_debit_170_87                    1  0.000000
17589_399_150_american express_237_credit_337_87       1  0.000000
17587_534_150_visa_226_debit_126_87                    1  0.000000
17586_481_150_visa_226_credit_337_87                   1  0.000000
17584_583_150_visa_226_credit

In [66]:
train_df.loc[tmp['card']=='9633_130_185_visa_138_debit_nan_nan',['TransactionDT']+['D%s'%(i+1) for i in range(14)]]

,TransactionDT,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14
2528,146900,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
2630,147946,171.0,NaN,116.0,0.0,NaN,0.0,NaN,254.708328,0.708333,0.0,NaN,0.0,254.0,NaN
3545,156574,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3738,158481,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4334,164526,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4383,165058,226.0,226.0,21.0,NaN,NaN,NaN,NaN,0.875000,0.875000,0.0,NaN,NaN,NaN,NaN
4431,165512,226.0,226.0,0.0,NaN,NaN,NaN,NaN,0.875000,0.875000,0.0,NaN,NaN,NaN,NaN
4909,170825,217.0,217.0,61.0,298.0,21.0,298.0,21.0,NaN,NaN,0.0,NaN,298.0,NaN,NaN
4928,171052,281.0,NaN,262.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4938,171128,281.0,NaN,0.0,NaN,NaN,NaN,NaN,0.958333,0.958333,0.0,NaN,NaN,NaN,NaN


In [74]:
train_df[['id_%s'%str(i+1).zfill(2) for i in range(38)]].head()

,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,...,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T
